In [1]:
from Bio.PDB import *
from utils.carb_utils import *
#import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix as dm


#-alternate_3_letter_codes pdb_sugar
#-out:level 100

#-beta
#-auto_detect_glycan_connections
#-alternate_3_letter_codes pdb_sugar



import os
import numpy as np
import pandas as pd
import copy

BOND_CUT = 1.75
INTERACT = 4.5

input_dir = "./"
os.chdir(input_dir)

In [2]:
ls = os.listdir('./pdb_pre/')
print(len(ls))

10736


In [3]:
def get_coor(structure):
    coor = []
    resid = []
    

    models = structure.get_models()
    models = list(models)
    if len(models) == 0:
        return [],[];
    
    for m in range(len(models)):

        chains = list(models[m].get_chains())
        for c in range(len(chains)):
            residues = list(chains[c].get_residues())
            for r in range(len(residues)):

                res = residues[r].get_resname()
                if res == 'HOH':
                    continue;

                atoms = list(residues[r].get_atoms())

                for a in range(len(atoms)):
                    at = atoms[a]

                    if 'H' in at.get_name():
                        continue;

                    #print(str(residues[r].get_parent().id).strip())

                    coor.append( at.get_coord() )
                    resid.append( [ str(at.get_name()), str(residues[r].id[1]).strip(), str(chains[c].id).strip(), str(residues[r].get_resname()) ] )
                      
        #print(len(coor))
        return np.array(coor), resid

In [4]:
def find_interactChains(coor,res,INTERACT=1.75):
    #determine chain-chain interactions
    d = dm(coor,coor) < INTERACT
    a = np.array( np.where(d == 1) )
    a = np.array(a)

    #chain_int = {}
    chain_int = ''
    for ii in range(a.shape[1]):

        res1 = res[ a[0,ii] ]
        res2 = res[ a[1,ii] ]

        #print(res1,res2)

        #chain_int[res1[1]] = res2[1]
        ope = res1[1] + ',' + res2[1]
        if ope not in chain_int:
            chain_int +=ope + '\n'

        #chain_int.update({res1[1]:res2[2]})

        #bad_c.append( res_c[a[0,ii]] )
        #print(res_c[a[0,ii]], res_p[a[1,ii]])

    #chain_int

    return chain_int

In [5]:
#enter coordinates and chain and get the polymer object instance
def chain_to_poly(my_chain,coor,res):
    """
    params:
        chain (str): chain identifier
        coor (arr n x 3): coordinates
        res (arr n x 4): residue information of each atom (aname, resnum, chain, resname)
    return:
        polymer
    """


    polymer = []
    c_resnum = -1;
    c_resname = ''
    n = []
    c = []
    
    
    for i in range(len(res)):
        ii = res[i]
        aname = ii[0]
        resnum = ii[1]
        chain = ii[2]
        resname = ii[3]
        
        

        if c_resnum != resnum:
            if c_resnum != -1 and len(n) > 1:
                #print(str(c_resnum),c_resname,np.array(c),n)
                m = mono(str(c_resnum),c_resname,np.array(c),n)
                polymer.append(m)
            #reset
            c_resnum = resnum;
            c_resname = resname
            n = []
            c = []

        if chain == my_chain:

            n.append(aname)
            c.append(coor[i])

    
    if len(n) > 1:
        m = mono(str(c_resnum),c_resname,np.array(c),n)
        polymer.append(m)

    return poly(polymer)

In [8]:
fasta = ''

parser=PDBParser()

tor_bois = []


for ii in range(len(ls)):
    pdb = ls[ii]
    
    if ii % 500 == 0:
        print(ii)
    
    if '_carb' not in pdb:
        continue;
        
    p = pdb.split('.')[0][:4]
    
    #if ii == 2:
    #    break;
    #try:
    structure=parser.get_structure("carb", 'pdb_pre/'+ pdb)
    print(ii,pdb)

    coor, res = get_coor(structure)
    if len(coor) == 0:
        print('nope')
        continue
    #inter = find_interactChains(coor,res)
    #print(inter)

    chains = []
    for ii in res:
        #print(ii)
        #break;
        if ii[2] not in chains:
            chains.append(ii[2])
    #print(chains)
    for c in chains:
        try:
            po = chain_to_poly(c,coor,res)
            #print(len(po))
            if len(po) > 1:
                tor_bois.append([pdb,c])

            np.save('npy_pre/' + p + '_' + c + '.npy',po)
        except:
            continue;

        #fasta += get_fasta(structure,p)
    #except:
        #bad.append(pdb)
    #    print('nope')
        
    #if ii >= 10:
    #    break;
    
    
#f = open('pdb_pre/all_fasta.fa','w+')
#f.write(fasta)
#f.close()

0
12 6YLJ_carb.pdb
16 4YY1_carb.pdb
24 5W11_carb.pdb
36 1QNR_carb.pdb
37 1OH3_carb.pdb
39 6LCR_carb.pdb
41 3ZOA_carb.pdb
43 3WMV_carb.pdb
44 4HG6_carb.pdb
45 4yef_carb.pdb
49 3WL0_carb.pdb
52 4m3m_carb.pdb
53 1HJV_carb.pdb
55 1GQJ_carb.pdb
59 2EVS_carb.pdb
64 1HJW_carb.pdb
65 1GQK_carb.pdb
67 3WL1_carb.pdb
68 3N9K_carb.pdb
71 7AIW_carb.pdb
72 6OR4_carb.pdb
76 2AEQ_carb.pdb
78 8abp_carb.pdb
81 2AEP_carb.pdb
87 3i8t_carb.pdb
90 4J2S_carb.pdb
91 1UIB_carb.pdb
98 3WDU_carb.pdb
100 5GAL_carb.pdb
101 2jdm_carb.pdb
103 5ZKZ_carb.pdb
105 4KBB_carb.pdb
106 3N0I_carb.pdb
107 3A4X_carb.pdb
108 7ER1_carb.pdb
109 3sww_carb.pdb
110 6E57_carb.pdb
111 1V6W_carb.pdb
118 3F5F_carb.pdb
119 3bkl_carb.pdb
122 5Z88_carb.pdb
123 1V6V_carb.pdb
129 6SZ6_carb.pdb
133 7DEQ_carb.pdb
134 4XEN_carb.pdb
135 1UU4_carb.pdb
139 3ehn_carb.pdb
141 4A05_carb.pdb
142 7C6K_carb.pdb
143 5KJQ_carb.pdb
145 3P5S_carb.pdb
147 2VGQ_carb.pdb
148 7XDQ_carb.pdb
149 1UU5_carb.pdb
151 3VL9_carb.pdb
152 5KK7_carb.pdb
153 1GZT_carb.pdb


In [9]:
f = open('pdb_pre/tor_bois.txt','w+')
f.write('pdb,chain\n')
for ii in tor_bois:
    f.write(ii[0] + ',' + ii[1] + '\n')
f.close()